In [3]:
import nltk
import pandas as pd

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SOORAJ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
fake = pd.read_csv('Fake.csv')
genuine = pd.read_csv('True.csv')

In [6]:
display(fake.head())

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [7]:
fake['genuineness'] = 0
genuine ['genuineness'] = 1

In [8]:
data = pd.concat([fake, genuine], axis = 0)

In [18]:
display(data)

,text,genuineness
0,"[Donald, Trump, just, couldn, t, wish, all, Am...",0
1,"[House, Intelligence, Committee, Chairman, Dev...",0
2,"[On, Friday, ,, it, was, revealed, that, forme...",0
3,"[On, Christmas, day, ,, Donald, Trump, announc...",0
4,"[Pope, Francis, used, his, annual, Christmas, ...",0
...,...,...
44893,"[BRUSSELS, (, Reuters, ), -, NATO, allies, on,...",1
44894,"[LONDON, (, Reuters, ), -, LexisNexis, ,, a, p...",1
44895,"[MINSK, (, Reuters, ), -, In, the, shadow, of,...",1
44896,"[MOSCOW, (, Reuters, ), -, Vatican, Secretary,...",1


In [10]:
data = data.reset_index(drop=True)

In [11]:
data = data.drop(['subject','date','title'],axis=1)

In [14]:
from nltk.tokenize import word_tokenize
data['text'] = data['text'].apply(word_tokenize)

TypeError: expected string or bytes-like object

In [23]:
from nltk.stem.snowball import SnowballStemmer
sb = SnowballStemmer("english", ignore_stopwords=False)

In [24]:
def stem_it(text):
    return [sb.stem(word) for word in text]

In [31]:
data['text'] = data['text'].apply(stem_it)

KeyboardInterrupt: 

In [26]:
def stopword_remove(t):
    return [word for word in t if len(word)>>2]

In [29]:
data ['text'] = data['text'].apply(stopword_remove)

In [30]:
data

,text,genuineness
0,"[Donald, Trump, just, couldn, wish, Americans,...",0
1,"[House, Intelligence, Committee, Chairman, Dev...",0
2,"[Friday, revealed, that, former, Milwaukee, Sh...",0
3,"[Christmas, Donald, Trump, announced, that, wo...",0
4,"[Pope, Francis, used, annual, Christmas, messa...",0
...,...,...
44893,"[BRUSSELS, Reuters, NATO, allies, Tuesday, wel...",1
44894,"[LONDON, Reuters, LexisNexis, provider, legal,...",1
44895,"[MINSK, Reuters, shadow, disused, Soviet-era, ...",1
44896,"[MOSCOW, Reuters, Vatican, Secretary, State, C...",1


In [33]:
data['text']= data['text'].apply(' '.join)

In [35]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train, y_test = train_test_split(data['text'],data['genuineness'], test_size=0.25)
display(X_train.head())
print('\n')
display(y_train.head())


37027    BRUSSELS Reuters European Union lawmakers have...
4513     member Donald Trump presidential campaign cree...
7646     know that there scientist creationist same tim...
29934    YORK Reuters Senate Republican leader Mitch Mc...
5145     More proof that Donald Trump presidential camp...
Name: text, dtype: object

37027    1
4513     0
7646     0
29934    1
5145     0
Name: genuineness, dtype: int64

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
my_tfidf = TfidfVectorizer (max_df=0.7)
tfidf_train = my_tfidf.fit_transform(X_train)
tfidf_test = my_tfidf.transform(X_test)

In [37]:
print(tfidf_train)

  (0, 89081)	0.03185772297036217
  (0, 45842)	0.07411030922920181
  (0, 60453)	0.05483419026446321
  (0, 68596)	0.04187457572588324
  (0, 68406)	0.058352318403985935
  (0, 75014)	0.03958299079165117
  (0, 11966)	0.057660684025409546
  (0, 80113)	0.04054883651767177
  (0, 68625)	0.02703275749062245
  (0, 71057)	0.04971817172474412
  (0, 49653)	0.031083207881509048
  (0, 61062)	0.025921675132305894
  (0, 88085)	0.032652576910930445
  (0, 75925)	0.026242251271444694
  (0, 35471)	0.043276979784246015
  (0, 61795)	0.1021334531364222
  (0, 81203)	0.07848382170044271
  (0, 27890)	0.03399994072380986
  (0, 88784)	0.06900313572142085
  (0, 57195)	0.0359485282318042
  (0, 94333)	0.04150097430680783
  (0, 14594)	0.022050072843148475
  (0, 78784)	0.047521367651869796
  (0, 73722)	0.03022403742112489
  (0, 47900)	0.04357597752007774
  :	:
  (33672, 78341)	0.04539172137295841
  (33672, 94647)	0.026616048680782968
  (33672, 17868)	0.056156113888924254
  (33672, 31803)	0.028756495385438046
  (33672, 8

In [38]:
from sklearn.linear_model import LogisticRegression
model_1 = LogisticRegression(max_iter=900)
model_1.fit(tfidf_train, y_train)

LogisticRegression(max_iter=900)

In [39]:
from sklearn.metrics import accuracy_score
pred_1 = model_1.predict(tfidf_test)
cr1 = accuracy_score(y_test, pred_1)
print(cr1*100)

98.89532293986638


In [40]:
from sklearn.linear_model import PassiveAggressiveClassifier

model = PassiveAggressiveClassifier(max_iter=50)
model.fit(tfidf_train, y_train)

PassiveAggressiveClassifier(max_iter=50)

In [41]:
y_pred = model.predict(tfidf_test)
accscore = accuracy_score(y_test, y_pred)
print('THE ACCURACY OF PREDICTION IS', accscore*100)

THE ACCURACY OF PREDICTION IS 99.66146993318486
